针对RPM产品，给定一个问题和解决方案，先扩张问题描述，然后反思这个更新的问题描述是否与原始版本保持一致。接着，根据更新的问题描述和原始的解决方案来生成多个解决方案，
再结合多个解决方案来生成最终的解决方案。目前的重点是完善问题描述和解决方案。

Use GPT-4

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


load_dotenv("../openai.env")
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0.7, max_tokens=3000)

In [2]:
import pandas as pd

ideas = {}
df = pd.read_excel('patents/rpm-patents-0627b.xlsx')
for i, row in df.iterrows():
    pno, question, solution = row['no.'], row['question'], row['solution']
    if pd.isna(question) or pd.isna(solution): continue
    ideas[pno] = {'question': question, 'solution': solution}
print(ideas.keys())

dict_keys(['p51', 'P60', 'P61'])


In [3]:
ideas['P61']

{'question': '在任务挖掘中，大部分网页都有独特的URL，不同URL路径代表不同的类型的网页，同一类型的网页通过ID来区分，但由于ID的形式多种多样，根据URL区分相事类型的URL并不容易。比如博客文章常用/blog/3832.html来表示，其中的3832就是文章ID，所有博客文章都是同样的URL路径，只是文章ID不同。这可以通过简单的正则来表示，但由于ID的表示方法有很多种，可以是纯数字，也可以是UUID，也可以是前缀加数字等。目前没有通用的方法来识别URL中的变化部分，进而实现相同类型URL的判断。',
 'solution': '基本思路是先识别URL中相同部分和变化部分，将变化部分转换为变量，获得URL模板。使用这个模板去匹配更多的URL，匹配到的URL即被视为同一类型。更进一步的，可以比较匹配到的URL对应的标题，若是标题有相同部分，则认为它们是同一类型的。'}

In [4]:
template_rewriteq = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
这个发明点属于流程挖掘产品，请根据发明人提供的简要描述和以下规则，重新描述专利发明点要解决的问题。
1，待解决的问题应用于什么样的业务场景？请给出一个简单且有代表性的例子，方便理解。
2，这类问题的一般解决思路是什么？这样的解决思路存在什么样的功能或者性能上的弱点或者缺陷？请给出技术上的理由。
3，为什么要解决这个问题？它有多大的现实价值？请结合业务场景和技术特征进行分析。

待解决问题的简单描述: {raw_question}

请根据以上规则要求，以更完整、更详细的方式重新描述要解决的问题：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [6]:
result = rewriteq_chain.run({'raw_question': question})
print(result)

InvalidRequestError: This model's maximum context length is 8192 tokens. However, you requested 8476 tokens (476 in the messages, 8000 in the completion). Please reduce the length of the messages or completion.

In [3]:
template_commentq = '''你是一个专业的流程挖掘产品经理，请评估更新版本的问题描述是否存在不完善的地方。
如果存在不完善的地方，请重新描述一下。如果没有，可以直接复用原来的问题描述。

原始版本的问题描述: {raw_question}
更新版本的问题描述: {question}

请在更新版本的基础上，回答以下问题：
1，更新版本是否表达了与原始版本相同的问题？如果有不一致的地方，请修正一下；
2，更新版本是否增加了更多的技术细节、应用场景，而不只是引用或者复述原始版本？如果未增加更多的细节，请补充；
3，更新版本是否解决了原始版本中未清晰表达的关键内容？如果没有很好的解决，请重新更正；
4，更新版本都包含了哪些核心概念和术语？请解释这些概念和术语，以及它们之间可能的逻辑关系。

对更新版本的评估（总长度不超过2000字）：'''

prompt_commentq = PromptTemplate(input_variables=['raw_question', 'question'], template=template_commentq)
commentq_chain = LLMChain(llm=llm, prompt=prompt_commentq, output_key='commentq')

In [4]:
template_candidates = '''你们是三个著名的程序员，请根据问题描述、有效性评估及用户提供的解决方案，各自提供一个初步的且具体创新性的解决方案。
解决方案要包含基本的解决思路，使用的算法和数据描述，以及具体的实现步骤。

待解决的问题: {question}
对该问题描述的有效性评估：{commentq}
用户提供的解决方案：{raw_solution}

请一步步思考，确保解决方案能够解决上述问题，并转换为具体的代码实现。
你们三个的解决方案分别是（总长度不超过3000字）：'''

prompt_candidates = PromptTemplate(input_variables=['question', 'commentq', 'raw_solution'], template=template_candidates)
candidates_chain = LLMChain(llm=llm, prompt=prompt_candidates, output_key='candidates')

In [5]:
template_rewritea = '''你是一个专业的流程挖掘领域的专利代理，你的任务是帮助专利发明人完善发明点设计。
请根据待解决问题的描述，自动生成的解决方案和用户提供的解决方案，重新组合生成一个可以作为专利发明点的解决方案。
这个解决方案需要具体、详细，能够转换为代码实现，还要包含以下内容：
1，它的基本思路是什么？用到了哪些成熟算法和数据？请具体分析这个方法如何解决问题的。
2，它与一般的解决方法有什么不同？不管是算法上的不同，应用场景的不同，还是使用的数据不同都可以。
3，这些不同点对应的优势和价值是什么？如果创新点的价值不高，可以通过什么样的方法来提高？
4，它的具体解决步骤是什么？可以综合多个解决方案中的实需现，但要保证每个步骤都是必须的，而且能够解决全部问题。

待解决问题的描述: {question}
自动生成的解决方案: {candidates}
发明人提供的解决方案：{raw_solution}

请根据上述方案，在发明人提供的解决方案基础上，生成一个具备创新性的解决方案（总长度不超过2000字）：'''

prompt_rewritea = PromptTemplate(input_variables=['question', 'candidates', 'raw_solution'], template=template_rewritea)
rewritea_chain = LLMChain(llm=llm, prompt=prompt_rewritea, output_key='solution')

In [6]:
template_background = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据以下待解决问题及解决方案的描述，起草专利的背景介绍部分。
专利的背景介绍要包含所解决问题所在的技术领域介绍，问题的简单介绍，以及这个问题所产生的不良影响。
待解决问题的描述: {question}
解决方案的描述: {solution}
请生成这个专利发明点的背景介绍（总长度不超过2000字）：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [7]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。
可使用以下问题来引导生成过程：

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求（总长度不超过4000字）：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [8]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。
可使用这些问题来引导生成过程：它的主要算法或者技术是什么？需要什么样的软硬件运行环境？是否有多种实现方式？每种实现方式最简单的形式是什么样的？

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明（总长度不超过2000字）：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [9]:
from langchain.chains import SequentialChain

def generate(pno, question, solution, version=1):
    result = overall_chain({'raw_question': question, 'raw_solution': solution})

    with open(f'patents/patent-rpm-{pno}v{version}.txt', 'w') as f:
        f.write('>>> original question: \n' + result['raw_question'])  
        f.write('\n\n>>> original solution: \n' + result['raw_solution'])
        f.write('\n\n>>> revised question: \n' + result['question'])
        f.write('\n\n>>> commented question: \n' + result['commentq'])
        f.write('\n\n>>> solution candidates: \n' + result['candidates'])
        f.write('\n\n>>> final solution: \n' + result['solution'])
        f.write('\n\n>>> background: \n' + result['background'])
        f.write('\n\n>>> claims: \n' + result['claims'])
        f.write('\n\n>>> explanation: \n' + result['explanation'])

overall_chain = SequentialChain(
    chains=[rewriteq_chain, commentq_chain, candidates_chain, rewritea_chain, background_chain, claims_chain, explanation_chain],
    input_variables=["raw_question", "raw_solution"],
    # Here we return multiple variables
    output_variables=["question", "commentq", "candidates", "solution", "background", "claims", "explanation"],
    verbose=True)

In [10]:
import pandas as pd


version = 3
df = pd.read_excel('patents/rpm-patents-0627b.xlsx')
for i, row in df.iterrows():
    pno, question, solution = row['no.'], row['question'], row['solution']
    if pd.isna(question) or pd.isna(solution): continue
    print(pno, question)
    generate(pno, question, solution, version)

p51 在流程挖掘和任务挖掘中，往往会遇到原始日志中包含了多个不同类型的业务的情况，这样，在显示流程图时就需要判断哪个业务更为重要，需要置于流程图中的中心位置。一般的方法是根据对应业务流程的出现次数，这样的选择有可能与业务人员理解的主流程并不一致。为了缩小这种不一致，有必要在选择主流程时考虑更多的相关因素。


> Entering new SequentialChain chain...


InvalidRequestError: This model's maximum context length is 8192 tokens. However, you requested 8442 tokens (442 in the messages, 8000 in the completion). Please reduce the length of the messages or completion.